# Exploratory Data Analysis (EDA)
## Masters in AI & ML Project

This notebook performs comprehensive exploratory data analysis on the dataset.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Set plot style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print('✓ Libraries imported successfully')

## 1. Load Dataset

In [ ]:
# Load your dataset
# Replace 'your_dataset.csv' with your actual file path
data_path = '../data/raw/your_dataset.csv'

# Uncomment the appropriate line based on your file type
# df = pd.read_csv(data_path)
# df = pd.read_excel(data_path)

# For demonstration, create a sample dataset
# Remove this when you have your own data
from sklearn.datasets import make_classification
X, y = make_classification(n_samples=1000, n_features=10, n_informative=8, 
                          n_redundant=2, random_state=42)
df = pd.DataFrame(X, columns=[f'Feature_{i+1}' for i in range(10)])
df['Target'] = y

print(f'Dataset loaded: {df.shape[0]} rows, {df.shape[1]} columns')

## 2. Dataset Overview

In [ ]:
# Display first few rows
print('\n📊 First 5 rows:')
df.head()

In [ ]:
# Dataset info
print('\n📋 Dataset Information:')
df.info()

In [ ]:
# Statistical summary
print('\n📈 Statistical Summary:')
df.describe()

## 3. Missing Values Analysis

In [ ]:
# Check for missing values
missing = df.isnull().sum()
missing_pct = (missing / len(df)) * 100

missing_df = pd.DataFrame({
    'Column': missing.index,
    'Missing_Count': missing.values,
    'Missing_Percentage': missing_pct.values
}).sort_values('Missing_Count', ascending=False)

print('\n❌ Missing Values:')
print(missing_df[missing_df['Missing_Count'] > 0])

# Visualize missing values
if missing.sum() > 0:
    plt.figure(figsize=(10, 6))
    sns.barplot(x=missing_pct.values, y=missing_pct.index)
    plt.xlabel('Percentage of Missing Values')
    plt.title('Missing Values by Column')
    plt.tight_layout()
    plt.show()
else:
    print('✓ No missing values found!')

## 4. Data Types and Unique Values

In [ ]:
# Analyze data types and unique values
dtype_df = pd.DataFrame({
    'Column': df.columns,
    'Data_Type': df.dtypes.values,
    'Unique_Values': [df[col].nunique() for col in df.columns],
    'Sample_Value': [df[col].iloc[0] for col in df.columns]
})

print('\n🔤 Data Types and Unique Values:')
dtype_df

In [ ]:
# Separate numerical and categorical columns
numerical_cols = df.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()

print(f'\n🔢 Numerical columns ({len(numerical_cols)}): {numerical_cols}')
print(f'\n📝 Categorical columns ({len(categorical_cols)}): {categorical_cols}')

## 5. Target Variable Analysis

In [ ]:
# Analyze target variable
# Adjust 'Target' to your actual target column name
target_col = 'Target'

if target_col in df.columns:
    print(f'\n🎯 Target Variable: {target_col}')
    print('\nValue Counts:')
    print(df[target_col].value_counts())
    
    print('\nDistribution:')
    print(df[target_col].value_counts(normalize=True) * 100)
    
    # Visualize target distribution
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # Count plot
    df[target_col].value_counts().plot(kind='bar', ax=ax1, color='skyblue')
    ax1.set_title(f'Target Variable Distribution')
    ax1.set_xlabel(target_col)
    ax1.set_ylabel('Count')
    ax1.tick_params(axis='x', rotation=45)
    
    # Pie chart
    df[target_col].value_counts().plot(kind='pie', ax=ax2, autopct='%1.1f%%')
    ax2.set_title(f'Target Variable Proportion')
    ax2.set_ylabel('')
    
    plt.tight_layout()
    plt.show()

## 6. Numerical Features Distribution

In [ ]:
# Distribution of numerical features
if len(numerical_cols) > 0:
    n_cols = 3
    n_rows = (len(numerical_cols) + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, n_rows * 4))
    axes = axes.flatten() if n_rows > 1 else [axes]
    
    for idx, col in enumerate(numerical_cols):
        if idx < len(axes):
            df[col].hist(bins=30, ax=axes[idx], edgecolor='black')
            axes[idx].set_title(f'Distribution of {col}')
            axes[idx].set_xlabel(col)
            axes[idx].set_ylabel('Frequency')
    
    # Hide empty subplots
    for idx in range(len(numerical_cols), len(axes)):
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.show()

## 7. Correlation Analysis

In [ ]:
# Correlation matrix
if len(numerical_cols) > 1:
    correlation = df[numerical_cols].corr()
    
    plt.figure(figsize=(12, 10))
    sns.heatmap(correlation, annot=True, fmt='.2f', cmap='coolwarm', 
                center=0, square=True, linewidths=1)
    plt.title('Correlation Matrix', fontsize=16)
    plt.tight_layout()
    plt.show()
    
    # Find highly correlated features
    high_corr = []
    for i in range(len(correlation.columns)):
        for j in range(i+1, len(correlation.columns)):
            if abs(correlation.iloc[i, j]) > 0.8:
                high_corr.append((correlation.columns[i], 
                                correlation.columns[j], 
                                correlation.iloc[i, j]))
    
    if high_corr:
        print('\n⚠️ Highly correlated features (|correlation| > 0.8):')
        for feat1, feat2, corr_val in high_corr:
            print(f'   {feat1} <-> {feat2}: {corr_val:.3f}')
    else:
        print('\n✓ No highly correlated features found')

## 8. Outlier Detection

In [ ]:
# Box plots for outlier detection
if len(numerical_cols) > 0:
    n_cols = 3
    n_rows = (len(numerical_cols) + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, n_rows * 4))
    axes = axes.flatten() if n_rows > 1 else [axes]
    
    for idx, col in enumerate(numerical_cols):
        if idx < len(axes):
            df.boxplot(column=col, ax=axes[idx])
            axes[idx].set_title(f'Box Plot - {col}')
            axes[idx].set_ylabel(col)
    
    # Hide empty subplots
    for idx in range(len(numerical_cols), len(axes)):
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.show()

## 9. Categorical Features Analysis

In [ ]:
# Analyze categorical features
if len(categorical_cols) > 0:
    for col in categorical_cols:
        print(f'\n📊 {col}:')
        print(df[col].value_counts())
        
        # Plot if reasonable number of categories
        if df[col].nunique() <= 20:
            plt.figure(figsize=(10, 5))
            df[col].value_counts().plot(kind='bar', color='coral')
            plt.title(f'Distribution of {col}')
            plt.xlabel(col)
            plt.ylabel('Count')
            plt.xticks(rotation=45, ha='right')
            plt.tight_layout()
            plt.show()
else:
    print('\nNo categorical features found')

## 10. Summary and Next Steps

In [ ]:
print('\n' + '='*60)
print('EXPLORATORY DATA ANALYSIS SUMMARY')
print('='*60)
print(f'\n📊 Dataset Shape: {df.shape}')
print(f'🔢 Numerical Features: {len(numerical_cols)}')
print(f'📝 Categorical Features: {len(categorical_cols)}')
print(f'❌ Missing Values: {df.isnull().sum().sum()}')
print(f'🔄 Duplicate Rows: {df.duplicated().sum()}')

print('\n' + '='*60)
print('NEXT STEPS')
print('='*60)
print('1. Handle missing values (if any)')
print('2. Address outliers (if necessary)')
print('3. Feature engineering and selection')
print('4. Encode categorical variables')
print('5. Scale/normalize features')
print('6. Split data into train/test sets')
print('7. Build and evaluate models')
print('\n✓ EDA Complete!')